In [1]:
import re
import cv2
import numpy as np
import PIL
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r'C:\Users\V.Ashok\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
image_path =  r'C:\Users\V.Ashok\Desktop\project.py\my1.jpeg'
image = cv2.imread(image_path)
im1=cv2.imread(image_path)
img=Image.open(image_path)
def color_analysis(image):
    
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    aadhaar_lower = np.array([0, 50, 50])
    aadhaar_upper = np.array([10, 255, 255])

    pan_lower = np.array([80, 50, 50])
    pan_upper = np.array([130, 255, 255])
    aadhaar_mask = cv2.inRange(hsv_image, aadhaar_lower, aadhaar_upper)
 #print(aadhaar_mask)
    pan_mask = cv2.inRange(hsv_image, pan_lower, pan_upper)


    aadhaar_pixels = cv2.countNonZero(aadhaar_mask)
    pan_pixels = cv2.countNonZero(pan_mask)

    if aadhaar_pixels > pan_pixels:
        
        dominant_color = "Aadhaar"
    else:
        dominant_color = "PAN"
    return dominant_color

def aadhaar():
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    denoised = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21) # Image denoising
    smoothed = cv2.GaussianBlur(denoised, (3, 3), 0) # Smoothing
    thresh = cv2.adaptiveThreshold(smoothed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,  cv2.THRESH_BINARY, 13, 3)
    text = pytesseract.image_to_string(thresh)
    text = re.sub(r'\s+', ' ', text)
#print(text)
    name_pattern = r'\b[A-Z][a-zA-Z]*(?:\s+[A-Z][a-zA-Z]*){2,}\b'
    names = re.findall(name_pattern, text)
    name = ' '.join(names) if names else None
#if name:
#name = ' '.join(re.findall(r'\b[A-Z]+\b', name))
    dob_pattern = r"(\d{2}/\d{2}/\d{4})"
    aadhaar_pattern = r"\b(\d{4}\s\d{4}\s\d{4})\b"
    dob_match = re.search(dob_pattern, text)
    aadhaar_match = re.search(aadhaar_pattern, text)
    dob = dob_match.group(1) if dob_match else None
    aad=aadhaar_match.group(0) if aadhaar_match else None
    result = color_analysis(image)
    print("Type of Card: ", result)
    print("Name:",name )
    print("Date of Birth:", dob)
    if "MALE" in text:
        print("Sex: MALE")
    elif "Male" in text:
        print("Sex: Male")
    elif "FEMALE" in text:
        print("Sex: Female")
    elif "Female" in text:
        print("Sex: Female")

    else:
        print("Sex: Others")
    print("Aadhaar Number:", aad)
    vid_pattern = r'VID : (\d{4} \d{4} \d{4} \d{4})'
    matches = re.findall(vid_pattern, text)
    vid = matches[0] if len(matches) > 0 else None
    print("VID:",vid)
    return
                                    
def pan():
    basewidth = 600
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img_resized = img.resize((basewidth, hsize), PIL.Image.LANCZOS)
                                   
            
         
# Display the resized image
#display(img_resized)
    extracted_texts=[]
# Process ROI and extract text
    def process_roi(image, roi):
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        denoised = cv2.fastNlMeansDenoising(gray, None, 10, 10, 22)
        smoothed = cv2.GaussianBlur(denoised, (3, 3), 0)
        thresh = cv2.adaptiveThreshold(smoothed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 13, 30)

        text = pytesseract.image_to_string(thresh)
 #print(text)
        return text
# Define the list of ROIs (top-left and bottom-right coordinates)
    rois = [
     [(14, 212), (307, 255)],
     [(15, 269), (316, 307)],
 # Add more ROIs here if needed
     ]
# Process each ROI and extract text
    for roi_coords in rois:
                                       
        (x1, y1), (x2, y2) = roi_coords
        if x1 >= x2 or y1 >= y2 or x2 > img_resized.size[0] or y2 > img_resized.size[1]:
                                       
            print("Invalid ROI coordinates:", roi_coords)
            continue
        roi_array = np.array(img_resized)
        roi = roi_array[y1:y2, x1:x2, :]
        text = process_roi(img_resized, roi)
 #print(text)
        extracted_texts.append(text)

        cv2.waitKey(0)
    cleaned_texts = []
    for extracted_text in extracted_texts:
        
 # Remove unwanted characters and newlines
        cleaned_text = re.sub(r'[^\w\s]', '', extracted_text).replace('\n', ' ')
        cleaned_texts.append(cleaned_text)
    name_lines = cleaned_texts[0].split('\n')
#print(name_lines)
    father_name_lines = cleaned_texts[1].split('\n')
#print(father_name_lines)
    name_lines = [line.strip() for line in name_lines if line.strip()]
    father_name_lines = [line.strip() for line in father_name_lines if line.strip()]
    name = name_lines[-1] if name_lines else None
    if name:
                             
        name = ' '.join(re.findall(r'\b[A-Z]+\b', name))
    father_name = father_name_lines[-1] if father_name_lines else None
    if father_name:
        father_name = ' '.join(re.findall(r'\b[A-Z]+\b', father_name))
    result = color_analysis(image)
    print("Type of Card: ", result)
    print("Name:", name)
    print("Father's Name:", father_name)
    im=cv2.imread(image_path)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    denoised = cv2.fastNlMeansDenoising(gray, None, 10, 10, 22) # Image denoising
    smoothed = cv2.GaussianBlur(denoised, (3, 3), 0) # Smoothing
    thresh = cv2.adaptiveThreshold(smoothed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,  cv2.THRESH_BINARY, 13, 3)
    text = pytesseract.image_to_string(thresh)

#cv2.imshow('Thresholded Image', thresh)
#print(text)
    dob_pattern = r"(\d{2}/\d{2}/\d{4})"
    pan_regex = r"([A-Z]{5}\d{4}[A-Z]{1})"
    dob_match = re.search(dob_pattern, text)
    pan_match=re.search(pan_regex,text)
    dob = dob_match.group(1) if dob_match else None
    pan = pan_match.group(1) if pan_match else None
    print("Date of Birth:", dob)
#print("Aadhaar Number:", aad)
    print("PAN number:",pan)
    return
if color_analysis(image)=="PAN":
     pan()
if color_analysis(image)=='Aadhaar':
     aadhaar()
#cv2.imshow('Thresholded Image', thresh)
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Type of Card:  Aadhaar
Name: Vangala Venkata Sai Laxman
Date of Birth: 14/01/2004
Sex: MALE
Aadhaar Number: ???? 2?1? ?78?
VID: 9179 5630 4032 1526
